In [1]:
@show VERSION

using BenchmarkTools

V = [rand(2, 3) for _ in 1:1000]

C1 = @btime cat($V...; dims=3)

C2 = @btime foldl((x, y) -> cat(x, y; dims=3), $V)

function aacat!(C, A::AbstractVector)
    for i in keys(A)
        C[axes(A[i])..., i] .= A[i]
    end
    C
end
function aacat!(C, A)
    for i in keys(A)
        C[axes(A[i])..., i.I...] .= A[i]
    end
    C
end
function aacat(A; A1 = A[begin])
    C = similar(A1, axes(A1)..., axes(A)...)
    aacat!(C, A)
end
C3 = @btime aacat($V)

C4 = similar(C3)
@btime aacat!($C4, $V)
aacat!(C4, V)

@show axes(C4)
@show C1 == C2 == C3 == C4;

VERSION = v"1.8.0-beta1"
  16.631 ms (10013 allocations: 13.97 MiB)
  5.526 ms (20648 allocations: 23.55 MiB)
  17.500 μs (2 allocations: 46.94 KiB)
  17.300 μs (0 allocations: 0 bytes)
axes(C4) = (Base.OneTo(2), Base.OneTo(3), Base.OneTo(1000))
C1 == C2 == C3 == C4 = true


In [2]:
using OffsetArrays
w = [OffsetArray(rand(2, 3), 0:1, -1:1) for _ in Iterators.product(1:3, 1:3)]
W = OffsetArray(w, 0:2, -2:0)

3×3 OffsetArray(::Matrix{OffsetMatrix{Float64, Matrix{Float64}}}, 0:2, -2:0) with eltype OffsetMatrix{Float64, Matrix{Float64}} with indices 0:2×-2:0:
 [0.884794 0.482408 0.445004; 0.0111522 0.761849 0.45306]   …  [0.0889951 0.00375174 0.205639; 0.874426 0.712647 0.77176]
 [0.407049 0.196206 0.612513; 0.970393 0.416169 0.0157348]     [0.331117 0.363294 0.299127; 0.925211 0.856703 0.897712]
 [0.953153 0.982571 0.14442; 0.444591 0.0592617 0.0602498]     [0.0652447 0.700463 0.787878; 0.893807 0.298732 0.0059241]

In [3]:
E = similar(W[begin], axes(W[begin])..., axes(W)...)
@btime aacat!($E, $W)

  255.091 ns (0 allocations: 0 bytes)


2×3×3×3 OffsetArray(::Array{Float64, 4}, 0:1, -1:1, 0:2, -2:0) with eltype Float64 with indices 0:1×-1:1×0:2×-2:0:
[:, :, 0, -2] =
 0.884794   0.482408  0.445004
 0.0111522  0.761849  0.45306

[:, :, 1, -2] =
 0.407049  0.196206  0.612513
 0.970393  0.416169  0.0157348

[:, :, 2, -2] =
 0.953153  0.982571   0.14442
 0.444591  0.0592617  0.0602498

[:, :, 0, -1] =
 0.499203   0.46028   0.504579
 0.0589405  0.589531  0.550563

[:, :, 1, -1] =
 0.381717  0.728156   0.207326
 0.850348  0.0680289  0.276464

[:, :, 2, -1] =
 0.361449  0.768525  0.334865
 0.23814   0.324103  0.655979

[:, :, 0, 0] =
 0.0889951  0.00375174  0.205639
 0.874426   0.712647    0.77176

[:, :, 1, 0] =
 0.331117  0.363294  0.299127
 0.925211  0.856703  0.897712

[:, :, 2, 0] =
 0.0652447  0.700463  0.787878
 0.893807   0.298732  0.0059241

In [4]:
@btime aacat($W)

  293.258 ns (2 allocations: 592 bytes)


2×3×3×3 OffsetArray(::Array{Float64, 4}, 0:1, -1:1, 0:2, -2:0) with eltype Float64 with indices 0:1×-1:1×0:2×-2:0:
[:, :, 0, -2] =
 0.884794   0.482408  0.445004
 0.0111522  0.761849  0.45306

[:, :, 1, -2] =
 0.407049  0.196206  0.612513
 0.970393  0.416169  0.0157348

[:, :, 2, -2] =
 0.953153  0.982571   0.14442
 0.444591  0.0592617  0.0602498

[:, :, 0, -1] =
 0.499203   0.46028   0.504579
 0.0589405  0.589531  0.550563

[:, :, 1, -1] =
 0.381717  0.728156   0.207326
 0.850348  0.0680289  0.276464

[:, :, 2, -1] =
 0.361449  0.768525  0.334865
 0.23814   0.324103  0.655979

[:, :, 0, 0] =
 0.0889951  0.00375174  0.205639
 0.874426   0.712647    0.77176

[:, :, 1, 0] =
 0.331117  0.363294  0.299127
 0.925211  0.856703  0.897712

[:, :, 2, 0] =
 0.0652447  0.700463  0.787878
 0.893807   0.298732  0.0059241